In [3]:
# Import all of the required modules to run the notebook 

from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u
from datetime import datetime

from sunpy.net import Fido, attrs as a
from sunpy.timeseries import TimeSeries

from stixpy.timeseries import quicklook # This registers the STIX timeseries with sunpy
from stixpy.net.client import STIXClient  # This registers the STIX client with Fido
from stixdcpy import auxiliary as aux
from stixpy.product import Product

In [ ]:
# The vast majority of the work in this notebook follows procedures further explained explained on 
# https://stixpy.readthedocs.io/en/latest/tutorials/quickstart.html
# This webpage should help with the excercises. 

In [34]:
# Use the stixdcpy package to access Solar Orbiter Emphemeris data which can be used to plot the spacecraft position 
# relative to the Earth and the Sun.  
emph=aux.Ephemeris.from_sdc(start_utc='2020-02-10T00:00:00', end_utc='2025-04-15T01:00:00', steps=100)

In [ ]:
# Use the automatic plotting function to plot the relative positions of Solar Orbiter, the Sun and Earth. 
# This is in the Heliocentric Earth ecliptic reference frame which explains why the tracked position does not appear at
# all physical. 
emph.peek()

In [ ]:
# 1. Use the method shown above the generate plots showing the location of Solar Orbiter for the following dates:
# 19th of March 2024 
# 10th of July 2023 


In [ ]:
# Here we will take a look at the quicklook data for the dates shown above and see if there are any interesting looking
# flares in the datasets. We will then focus on the 10th of July 2023, to download the science data, look at the 
# lightcurves and create some spectra which we will also background subtract. This will give insights into how the 
# spectra change over the course of a solar flare. This can help us to understand the physical processes that are 
# driving the flare process. 

In [ ]:
# 2. Use the example below to take a look at the quicklook data for 19th of March 2024 and try to zoom in on the
# interesting looking time ranges. 

ql_query = Fido.search(a.Time('2024-03-19T00:00:00', '2024-03-19T23:59:00',), a.Instrument.stix,
                    a.stix.DataProduct.ql_lightcurve)

ql_query



In [ ]:
ql_files = Fido.fetch(ql_query)

In [ ]:
ql_lightcurves = TimeSeries(ql_files)

ql_lightcurves.peek()

In [ ]:
# This is an example of an interesting flare, however there are many on this particular day. Zoom in on different
# regions using this method and see that the flares can differ drastically in their appearance and evolution. A 
# response in the higher energy bands represents non-thermal processes. 

ql_lightcurves.plot()
plt.xlim(datetime(2024, 3, 19, 23, 10, 0), datetime(2024, 3, 19, 23, 59, 0))

In [ ]:
# 2. Now repeat this for 10th of July 2023 and try to zoom in on the
# interesting looking time ranges. The flare we shall focus on is between 23:00 (10/07/23) and 02:00 (11/07/23)
# To look at this flare, as it crosses over a date, you will have to combine two quicklook files. Use the hint below.

# combination of files

combined_ts = ql_lightcurves[0]
for lc in ql_lightcurves[1:]:
    combined_ts = combined_ts.concatenate(lc)

combined_ts.peek()

In [ ]:
# 3. Insert the correct times for the flare we selected above on 10/07/23.

# Now we will download the science data for the interesting event found above for 10th July 2023. Use the code below 
# to insert a roughly 3 hour time window around the interesting event identified above in the format:
# 'yyyy-mm-ddThh:mm:ss'. 

sci_query = Fido.search(a.Time('yyyy-mm-ddThh:mm:ss', 'yyyy-mm-ddThh:mm:ss'), a.Instrument.stix, a.stix.DataType.sci)
sci_query

In [ ]:
# Here I have selected the correct data file from the list of results generated above. 

sci_query[0][[4]]

In [ ]:
# Download the correct file. 

sci_files = Fido.fetch(sci_query[0][[4]])

In [ ]:
# We need to convert the file to a Product so that it can be manipulated.

spec = Product(sci_files[0])
spec

In [ ]:
# https://stixpy.readthedocs.io/en/latest/tutorials/quickstart.html

# 4. Check the online documentation of the stixpy python package for three automatic plotting routines, they are all
# named plot_... Use these to create quick plots of the data in the science file. The two most important plots will
# be the lightcurve plot and the spectrogram plot. 

In [ ]:
# quick plot 1

In [ ]:
# quick plot 2

In [ ]:
# quick plot 3

In [ ]:
# Here we extract the full dataset from the science file. 
# the function get_data() and its properties can be understood more fully by taking a look at the following webpage
# https://stixpy.readthedocs.io/en/latest/api/stixpy.product.sources.CompressedPixelData.html

counts_sci_full, errors_sci_full, times_sci_full, deltatimes_sci_full, energies_sci_full = spec.get_data()

pprint(energies_sci_full)


In [ ]:
# 5. Insert the indices to sum between in order to generate lightcurves of differing energy ranges. Try to produce 
# a plot with three lightcurves, one for 4-10 Kev, 10-15 keV, 15-25 keV and 25-50keV. 
# Hint: Use the energy bin indices printed above.   

spec.plot_timeseries(energy_indices=[[,],[,],[,],[,]])

In [ ]:
# Here we use get_data to extract the data from the science file. You can play around with the different selections for
# pixels, times and detectors. Here we sum all of the time indices, pixel indices and detector indices. 
# The aim is to produce a single spectrum for the entire flare duration.  

counts_sci, errors_sci, times_sci, deltatimes_sci, energies_sci = spec.get_data(pixel_indices=[[0,7]],
                                                                                time_indices=[[0,len(times_sci_full)-1]],
                                                                                detector_indices=[[0,22]])
# Extract the data into a more user friendly format

counts_sci = counts_sci[0][0][0]
errors_sci = errors_sci[0][0][0]

In [ ]:
# Take a look at the counts and notice the units!! Is this an absolute value, or a count rate?
counts_sci

In [ ]:
# Each energy bin has a lower bin edge and an upper bin edge. We need to calculate the bin centers so that we can plot 
# uncertainties at the bin centers. 

energy_edges = np.concatenate([energies_sci['e_low'].value,[energies_sci['e_high'][-1].value]])*u.keV
energy_centers = energies_sci['e_low'] + 0.5*np.diff(energy_edges)

# Notice that we always plot in loglog space! 

plt.figure()
plt.loglog()
plt.errorbar(energy_centers,counts_sci,yerr=errors_sci,marker=None,capsize=2,linestyle='', color='red')
plt.stairs(counts_sci,energy_edges,baseline=None, color='red')
plt.xlim([3.5,52])

# Here we have a great spectrum of the full event. However this spectrum is not background subtracted. 

In [ ]:
# Now we need to find and download a suitable background file to subtract the background flux from the science 
# data flux, allowing us to see the flux produced solely by the flare.

In [ ]:
bkg_query = Fido.search(a.Time('2023-07-06T20:00:00', '2023-07-06T22:00:00'), a.Instrument.stix, a.stix.DataType.sci)
bkg_query


In [ ]:
bkg_files = Fido.fetch(bkg_query[0][[0]])

In [ ]:
bkg = Product(bkg_files[0])
bkg

In [ ]:
# 6. Plot the quicklook spectrogram plot and compare it with that of the science data. The other two plots are not 
# important here. 


In [17]:
# 7. Use the get_data function to retrive all of the data. I suggest using the names 
# counts_bkg_full, errors_bkg_full, times_bkg_full, deltatimes_bkg_full, energies_bkg_full 


In [ ]:
# 8. Now repeat the above step using get_data function to retrive the relevant data for background subtraction. 
# I suggest using the names 
# counts_bkg, errors_bkg, times_bkg, deltatimes_bkg, energies_bkg  
# You will need to insert the correct time and energy_indices. 
# Hints: 
# 1. You need to select all of the time indices as with the example above
# 2. You need to select the corresponding energy bins to that of the science data, print out the bins as we did for the 
# science data and set the 'start' and 'stop' indices to those which match the energies included in the science data. 


counts_bkg, errors_bkg, times_bkg, deltatimes_bkg, energies_bkg = bkg.get_data(pixel_indices=[[0,7]],
                                                                                time_indices=[[,]],
                                                                                detector_indices=[[0,22]],
                                                                                energy_indices=np.arange('start','stop',1))

counts_bkg= counts_bkg[0][0][0]
errors_bkg= errors_bkg[0][0][0]

energies_bkg

In [ ]:
# 9. Create a plot for the background as we did above for the science data. Note the plotting scale!


In [ ]:
# 10. Here I have written the code to subract the background and have added the errors in quadrature. Write the plotting
# code to take a look at the result. 

counts_bkg_subtracted = counts_sci - counts_bkg
errors_bkg_subtracted = np.sqrt(errors_bkg**2 + errors_sci**2)


In [ ]:
# 11. The plots you have created so far have been for the entire dataset, however we would normally want to view the 
# evolution of the flare. Write some code to extract the science data for shorter time bins to see the evolution of the
# shape of the spectrum across the flare duration. Notice how the shape changes. In the spectra where there are more 
# counts at higher energies, we are seeing the nonthermal phase of the flare.

# Hints:
# 1. The only argument that needs to be amended is time_indices. 
# 2. The background can be used as it is, as the result is a count rate and so is time averaged, meaning we don't need
# to change the background at all and can subtract this from every time bin we create. 
# 3. To improve the appearance of the plot we may want to ignore any data below a certain value such as 
# 10^{-1} ct kev^{-1} s^{-1} which may be spurious due to imperfect background subtraction.